# SPRINT16　公開されている実装を動かす。

##  グーグルコラボによる学習。  KaggleからSimpsons Dataset をダウンロードしてFaster RCNNなどをグーグルコラボ上で稼働させる。

##  【問題１】　学習と推定

In [0]:
import tensorflow

In [0]:
tensorflow.VERSION

'1.15.0'

In [0]:
!pip install kaggle

## ＜備忘録＞ マイドライブへのマウンド（リンク先からログインコードを取得する）

In [0]:
# 自分のマイドライブにマウントする

from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
mkdir .kaggle

In [0]:
ls -a

./  ../  .config/  drive/  .kaggle/  sample_data/


In [0]:
#cd drive/"My Drive"

In [0]:
# https://www.kaggle.com/　へアクセスし、ログイン後に右上のプロフィール画像をクリック、さらに"My Account"をクリックする
# このサイトの「API」項目にある"Create New API Token"をクリックすると、kaggle.jsonファイルが自動的にダウンロードされる
# ローカルで、kaggle.json（ダウンロードフォルダにあるはず）をエディターで開く。
# このセルの以下のコードにある token = {'username':'***','key':'***'} における「***」部分を、
# ダウンロードしたkaggle.jsonを参照して書き換え、このセルを実行する

import json

token = {"username":"goemon024","key":"6f9946e0b40175e286a2e6126dabf5a5"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!chmod 600 /content/.kaggle/kaggle.json

In [0]:
pwd

'/content/drive/My Drive'

In [0]:
ls -a .kaggle

./  ../  kaggle.json


In [0]:
mkdir　/root/.kaggle/

In [0]:
!cp -R /content/.kaggle/kaggle.json /root/.kaggle/

In [0]:
!ls -a /root/.kaggle/

.  ..  kaggle.json


In [0]:
# kaggle APIコマンドが使えるかどうかを確認


!kaggle competitions list

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [0]:
# カレントディレクトリの変更

import os

os.chdir('/content/drive/My Drive/')

# カレントディレクトリの取得
print(os.getcwd())

/content/drive/My Drive


In [0]:
# simpsonデータセットのAPIコマンドでデータセットをダウンロードする
# APIコマンドはこちらのサイトを参照。https://www.kaggle.com/alexattia/the-simpsons-characters-dataset
# ＜メモ＞下記の末尾に　--force をつけると、強制的に実行せよ。となる。

!kaggle datasets download -d alexattia/the-simpsons-characters-dataset

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [0]:
## ＜メモ＞My Drive の下に　Simpsonsというドライブを作って、unzipunzipされたファイルを格納する。
!unzip /content/drive/'My Drive'/the-simpsons-characters-dataset.zip  -d Simpsons

ストリーミング出力は最後の 5000 行に切り捨てられました。
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1271.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1272.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1273.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1274.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1275.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1276.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1277.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1278.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1279.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1280.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1281.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset

## GitHub上で公開されているFasterRCNNを動かす（READMEファイルを参考にする）。

In [1]:
!git clone https://github.com/duckrabbits/ObjectDetection.git

Cloning into 'ObjectDetection'...


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# https://github.com/duckrabbits/ObjectDetection/tree/master
# この後、上述のgithubのreadmeに沿って、simpsons_datasetフォルダをプロジェクトフォルダ配下に格納しましょう。
# testデータなども同様に準備してみよう。

In [0]:
# カレントディレクトリの変更

import os

os.chdir('/content/drive/My Drive/')

# カレントディレクトリの取得
print(os.getcwd())

/content/drive/My Drive


##  プロジェクトフォルダ（ObjectDetection）下に、simpsons_datasetなどを移動させる。

In [0]:
!mv Simpsons/simpsons_dataset ObjectDetection/

In [0]:
!mv Simpsons/annotation.txt ObjectDetection/

In [0]:
## 課題では、githubのFasterRCNNのファイルを、スラックからのFasterRCNNファイルに置き換え

In [0]:
import os

os.chdir('/content/drive/My Drive/ObjectDetection')
print(os.getcwd())

/content/drive/My Drive/ObjectDetection


In [0]:
!mv simpsons_dataset1/simpsons_dataset ObjectDetection/

##  学習  
以下のコマンドを実行してください。  
python train.py -p annotation.txt  
以下のように設定ファイルのパスが出力されるので記録してください。推定時に使います。  
""" path to config file : ./save/train_20190309-220050_config.pickle """  
エポック数なども引数で変更できます。詳細はtrain.pyを参照してください。  

In [0]:
!python train.py -p annotation.txt

Using TensorFlow backend.
Parsing annotation files
Training images per class (19 classes) :
{'abraham_grampa_simpson': 687,
 'apu_nahasapeemapetilon': 206,
 'bart_simpson': 650,
 'bg': 0,
 'charles_montgomery_burns': 650,
 'chief_wiggum': 209,
 'comic_book_guy': 208,
 'edna_krabappel': 212,
 'homer_simpson': 718,
 'kent_brockman': 213,
 'krusty_the_clown': 429,
 'lisa_simpson': 756,
 'marge_simpson': 629,
 'milhouse_van_houten': 210,
 'moe_szyslak': 403,
 'ned_flanders': 675,
 'nelson_muntz': 219,
 'principal_skinner': 614,
 'sideshow_bob': 201}
-------------------------------
path to config file : ./save/train_20200318-070110_config.pickle
-------------------------------









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
i

##  推定  
以下のコマンドを実行してください。  
python predict.py -i [推定したい画像を入れたディレクトリのパス] -c [学習時に保存された設定ファイルのパス]  

In [0]:
!python predict.py -i Simpsons/kaggle_simpson_testset/kaggle_simpson_testset -c save/train_20200318-070110_config.pickle

Using TensorFlow backend.










2020-03-18 07:14:11.318216: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2020-03-18 07:14:11.318646: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x9709d40 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-03-18 07:14:11.318683: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-03-18 07:14:11.320742: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-03-18 07:14:11.454335: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-03-18 07:14:11.454866: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xa8c0e00 initialized for platform CUDA (this does not guarantee that XLA will be

In [0]:
####### セッションが途切れた場合の復旧処理  ##########
# 自分のマイドライブにマウントする
# リンクをクリックしてトークンをコピペ。
# カレントディレクトリの変更やギットクローン。
# これによりObjectDetection下に各種ファイルを再現する。

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import os

os.chdir('/content/drive/My Drive/ObjectDetection')
print(os.getcwd())

/content/drive/My Drive/ObjectDetection


## 【問題２】　コードリーディング

・faseter_rcnn.pyについて  
　RPNとClassifierがそれぞれKERASのSEAQUENTIALモデルによって定義されている。RPNとClassifierにおける畳み込みCNNにはresnetが使用される。  ClassifierにはROIからのインプットがある。  
・resnet.pyについて  
　RPNとClassifierが関数にて定義されている。これらは特徴マップを共有する（shared_layers）が、shared_layersはnn_base関数にて生成される。  
・roi_helpers.pyについて
　ROIプーリングは、rpn_to_roiの関数に対応している。

##  【問題３】学習済みの重みによる推定  

##  githubからクローニングしてyoro_v3を導入する。readmeに従って画像を出力する（添付写真）。

In [3]:
!git clone https://github.com/qqwweee/keras-yolo3

Cloning into 'keras-yolo3'...


In [0]:
import os

os.chdir('/content/drive/My Drive/keras-yolo3')
print(os.getcwd())

/content/drive/My Drive/keras-yolo3


In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights

In [0]:
!python convert.py yolov3.cfg yolov3.weights model_data/yolo.h5

Using TensorFlow backend.
Loading weights.
Weights Header:  0 2 0 [32013312]
Parsing Darknet config.
Creating Keras model.


Parsing section net_0
Parsing section convolutional_0
conv2d bn leaky (3, 3, 3, 32)




2020-03-19 13:21:36.895680: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2020-03-19 13:21:36.897825: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2398bc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-03-19 13:21:36.897875: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-03-19 13:21:36.902666: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-03-19 13:21:37.089477: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node

In [4]:
!python yolo_video.py --image
# コマンド入力後に現れるブランクに、ファイル名を入力するとＹＯＬＯによる出力が得られる。

python: can't open file 'yolo_video.py': [Errno 2] No such file or directory


## 【問題4】学習のためのファイルを作成  
## 【問題5】学習が行えることの確認
・シンプソンズのAnottation.txtを、YOLOのREADMEに記載の形式に書き換え（添付ファイル）。  
・書き換えたファイルの名前をtrain.txtに変更して、オブジェクトフォルダにアップロード。  
・ python convert.py -w yolov3.cfg yolov3.weights model_data/yolo_weights.h5を実行  
・train 実行（!python train.py）

＜備忘録＞　train実行時、module 'keras.backend' has no attribute 'control_flow_ops' のエラーが出たので下記のように変更。  
https://github.com/qqwweee/keras-yolo3/issues/544  
keras や tensorflow のバージョンなどに問題がある様子。  
・　!pip install tensorflow==1.4　でバージョンを変更（この修正では修復しなかった）。  
・　to /usr/local/lib/python3.6/dist-packages/keras/backend/__init__.pyのファイルに、  
from .load_backend import control_flow_ops  
from .load_backend import set_image_dim_ordering  
の２行を加筆（これによりtrain実行）。